In [1]:
import os

# change to target directory
os.chdir(r'C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1')

# Folders to read
folders = ["BHA", "CCA", "FAHA"]

# Use current working directory (since folders are next to your notebook)
base_dir = os.getcwd()

# Loop through folders and list all files
for folder in folders:
    folder_path = os.path.join(base_dir, folder)
    if os.path.exists(folder_path):
        print(f"\n📁 Files in folder: {folder}")
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                print(file_path)
    else:
        print(f"\n⚠️ Folder not found: {folder}")



📁 Files in folder: BHA
C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\BHA\BlueMesa.Elev.txt
C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\BHA\BlueMesa.Inflow.txt
C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\BHA\BlueMesa.Outflow.txt
C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\BHA\BlueMesa.Storage.txt
C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\BHA\ComputedStateDepletions.ArizonaActual.txt
C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\BHA\ComputedStateDepletions.CaliforniaActual.txt
C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\BHA\ComputedStateDepletions.ColoradoActual.txt
C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\BHA\ComputedStateDepletions.NevadaActual.txt
C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\BHA\ComputedStateDepletions.NewMexicoActual.t

In [2]:
import os
import re
import shutil

# === SETTINGS ===============================================================
# Folders containing your scenario files (must be siblings of this script/notebook)
SCENARIO_FOLDERS = ["BHA", "CCA", "FAHA"]

# Where to create the grouped folders (relative to the current working dir)
GROUP_ROOT = "CommonFiles"

# Only copy .txt files? (set to None to include all files)
EXT_WHITELIST = {".txt"}

# Copy vs move the renamed files into CommonFiles.
#   False -> COPY (safer; originals remain)
#   True  -> MOVE (files will be moved out of their scenario folders)
MOVE_INSTEAD_OF_COPY = False
# ===========================================================================

base_dir = os.getcwd()
group_root_dir = os.path.join(base_dir, GROUP_ROOT)
os.makedirs(group_root_dir, exist_ok=True)

# iclude only files with whitelisted extensions
def should_include(path):
    if not os.path.isfile(path):
        return False
    if EXT_WHITELIST is None: # include all files
        return True
    _, ext = os.path.splitext(path)
    return ext.lower() in EXT_WHITELIST

# add scenario prefix if missing
def ensure_prefixed(scenario, filename):
    """Return a filename that starts with '<scenario>_' (adds if missing)."""
    if filename.startswith(f"{scenario}_"):
        return filename
    return f"{scenario}_{filename}"

def base_key_without_scenario(scenario, prefixed_filename):
    """
    Given a filename like 'BHA_Mead.Elev.txt', return the grouping key 'Mead.Elev'
    (i.e., remove '<scenario>_' and the extension).
    """
    name = re.sub(rf"^{re.escape(scenario)}_", "", prefixed_filename)
    stem, _ = os.path.splitext(name)  # keep dots inside the stem (e.g., 'Mead.Elev')
    return stem

for scenario in SCENARIO_FOLDERS:
    scenario_dir = os.path.join(base_dir, scenario)
    if not os.path.isdir(scenario_dir):
        print(f"⚠️  Missing folder: {scenario_dir} (skipping)")
        continue

    print(f"\n📁 Processing: {scenario}")
    for fname in os.listdir(scenario_dir):
        src_path = os.path.join(scenario_dir, fname)
        if not should_include(src_path):
            continue

        # 1) Rename to ensure '<scenario>_' prefix
        prefixed_name = ensure_prefixed(scenario, fname)
        prefixed_path = os.path.join(scenario_dir, prefixed_name)
        if os.path.exists(prefixed_path):
            os.remove(prefixed_path)
        os.rename(src_path, prefixed_path)
        print(f"   Renamed (overwrote) -> {prefixed_name}")


        # 2) Group by common metric (e.g., 'Mead.Elev')
        key = base_key_without_scenario(scenario, os.path.basename(prefixed_path))
        dest_dir = os.path.join(group_root_dir, key)
        os.makedirs(dest_dir, exist_ok=True)

        dest_path = os.path.join(dest_dir, os.path.basename(prefixed_path))
        if MOVE_INSTEAD_OF_COPY:
            # Move (will remove from scenario folder)
            if os.path.abspath(prefixed_path) != os.path.abspath(dest_path):
                if os.path.exists(dest_path):
                    # Avoid clobbering: if same content, you can skip; otherwise, raise
                    raise FileExistsError(f"Destination exists: {dest_path}")
                shutil.move(prefixed_path, dest_dir)
                print(f"   Moved -> {dest_path}")
        else:
            # Copy (leave original in place)
            if not os.path.exists(dest_path):
                shutil.copy2(prefixed_path, dest_path)
                print(f"   Copied -> {dest_path}")
            else:
                print(f"   Skipped (already exists) -> {dest_path}")
                shutil.copy2(prefixed_path, dest_path)
                print(f"   Overwritten -> {dest_path}")

print("\n✅ Done.")
print(f"Grouped folders are under: {group_root_dir}")
print("Tip: set MOVE_INSTEAD_OF_COPY = True if you want to move instead of copy.")



📁 Processing: BHA
   Renamed (overwrote) -> BHA_BlueMesa.Elev.txt
   Copied -> C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\CommonFiles\BlueMesa.Elev\BHA_BlueMesa.Elev.txt
   Renamed (overwrote) -> BHA_BlueMesa.Inflow.txt
   Copied -> C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\CommonFiles\BlueMesa.Inflow\BHA_BlueMesa.Inflow.txt
   Renamed (overwrote) -> BHA_BlueMesa.Outflow.txt
   Copied -> C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\CommonFiles\BlueMesa.Outflow\BHA_BlueMesa.Outflow.txt
   Renamed (overwrote) -> BHA_BlueMesa.Storage.txt
   Copied -> C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\CommonFiles\BlueMesa.Storage\BHA_BlueMesa.Storage.txt
   Renamed (overwrote) -> BHA_ComputedStateDepletions.ArizonaActual.txt
   Copied -> C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\CommonFiles\ComputedStateDepletions.ArizonaActual\BHA_ComputedStateDepletion

In [3]:
import os
import re
import pandas as pd

# ===== Settings ==========================================================
SCENARIO_FOLDERS = ["BHA", "CCA", "FAHA"]
BASE_DIR = os.getcwd()  # run this where the scenario folders live
ONLY_EXT = {".txt"}     # set to None to include all extensions
# ========================================================================

def read_text(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def parse_values_after_series_slot(text):
    """
    Find '# Series Slot:' header and parse all subsequent numeric lines.
    """
    m = re.search(r"^\s*#\s*Series\s+Slot:.*?(?:\r?\n)(.*)$", text, flags=re.S | re.M)
    if not m:
        return []
    vals = []
    for line in m.group(1).splitlines():
        s = line.strip()
        if not s or s.startswith("#"):
            continue
        try:
            vals.append(float(s))
        except ValueError:
            pass
    return vals

def parse_dates(text):
    """
    Pull start_date and end_date (YYYY-MM-DD). Either may be missing.
    """
    start_match = re.search(r"start_date:\s*(\d{4}-\d{2}-\d{2})", text)
    end_match   = re.search(r"end_date:\s*(\d{4}-\d{2}-\d{2})", text)
    start_date = pd.to_datetime(start_match.group(1)) if start_match else None
    end_date   = pd.to_datetime(end_match.group(1)) if end_match else None
    return start_date, end_date

def months_inclusive(s, e):
    """Inclusive month count between two dates (e.g., Jan->Jan = 1)."""
    return (e.year - s.year) * 12 + (e.month - s.month) + 1

def years_inclusive(s, e):
    """Inclusive year count between two dates (e.g., 2000->2000 = 1)."""
    return (e.year - s.year) + 1

def infer_freq(start_date, end_date, n):
    """
    Infer 'MS' (monthly) or 'AS' (annual) given start/end and number of values.
    Priority: exact match; fallback to heuristics if only start_date is present.
    """
    if start_date is not None and end_date is not None:
        m = months_inclusive(start_date, end_date)
        y = years_inclusive(start_date, end_date)
        if m == n:
            return "MS"
        if y == n:
            return "AS"
        # Sometimes files include end_date but code historically ignores it; try nearest
        # If n is close (within 1) to months, call it monthly; if close to years, annual
        if abs(m - n) <= 1:
            return "MS"
        if abs(y - n) <= 1:
            return "AS"
    # Fallbacks without reliable end_date: many CRSS outputs are monthly
    if n >= 24:         # long sequences usually monthly
        return "MS"
    if n <= 3:          # very short sequences likely annual
        return "AS"
    # Default bias to monthly
    return "MS"

def build_index(start_date, freq, n):
    """
    Build a DatetimeIndex of length n for the inferred frequency.
    - Monthly: anchor exactly at start_date month ("MS")
    - Annual:  anchor at Jan 1 of start_date's year ("AS")
    """
    if freq == "MS":
        if start_date is None:
            raise ValueError("Monthly series needs start_date to anchor.")
        start = pd.Timestamp(start_date.year, start_date.month, 1)
        return pd.date_range(start=start, periods=n, freq="MS")
    elif freq == "AS":
        if start_date is None:
            raise ValueError("Annual series needs start_date to anchor.")
        start = pd.Timestamp(start_date.year, 1, 1)
        return pd.date_range(start=start, periods=n, freq="AS")
    else:
        raise ValueError(f"Unsupported freq: {freq}")

def should_include(path):
    if not os.path.isfile(path):
        return False
    if ONLY_EXT is None:
        return True
    _, ext = os.path.splitext(path)
    return ext.lower() in ONLY_EXT

# ===== Process each scenario folder ======================================
for folder in SCENARIO_FOLDERS:
    folder_path = os.path.join(BASE_DIR, folder)
    if not os.path.isdir(folder_path):
        print(f"⚠️  Missing folder: {folder_path} (skipping)")
        continue

    monthly_series = {}
    annual_series = {}
    summary_rows = []

    for filename in sorted(os.listdir(folder_path)):
        if not filename.endswith(".txt"):
            continue
        path = os.path.join(folder_path, filename)
        if not should_include(path):
            continue

        text = read_text(path)
        values = parse_values_after_series_slot(text)
        if not values:
            # skip files that don't match the expected format
            summary_rows.append((filename, "N/A", "No data parsed"))
            continue

        start_date, end_date = parse_dates(text)
        freq = infer_freq(start_date, end_date, len(values))

        # Construct index
        try:
            idx = build_index(start_date, freq, len(values))
        except Exception as e:
            summary_rows.append((filename, "Unknown", f"Index error: {e}"))
            continue

        series_name = os.path.splitext(filename)[0]  # keep dots in base name

        if freq == "MS":
            monthly_series[series_name] = pd.Series(values, index=idx)
            summary_rows.append((filename, "Monthly", f"{len(values)} points from {idx[0].date()}"))
        elif freq == "AS":
            annual_series[series_name] = pd.Series(values, index=idx)
            summary_rows.append((filename, "Annual", f"{len(values)} points from {idx[0].date()}"))
        else:
            summary_rows.append((filename, "Unknown", f"{len(values)} points"))

    # Assemble DataFrames (align on time)
    monthly_df = pd.DataFrame(monthly_series).sort_index() if monthly_series else pd.DataFrame()
    annual_df  = pd.DataFrame(annual_series).sort_index()  if annual_series  else pd.DataFrame()

    # Save outputs
    out_monthly = os.path.join(BASE_DIR, f"{folder}_monthly_timeseries.csv")
    out_annual  = os.path.join(BASE_DIR, f"{folder}_annual_timeseries.csv")
    if not monthly_df.empty:
        monthly_df.to_csv(out_monthly)
    if not annual_df.empty:
        annual_df.to_csv(out_annual)

    # Print summary
    print(f"\n=== {folder} ===")
    print(f"Monthly series: {len(monthly_series)}  |  Annual series: {len(annual_series)}")
    for fname, step, note in summary_rows:
        print(f"- {fname}: {step}  ({note})")

    if not monthly_df.empty:
        print(f"→ Saved monthly CSV: {out_monthly}  (shape={monthly_df.shape})")
    if not annual_df.empty:
        print(f"→ Saved annual  CSV: {out_annual}   (shape={annual_df.shape})")

print("\n✅ Done. Each folder produced up to two CSVs: '<FOLDER>_monthly_timeseries.csv' and '<FOLDER>_annual_timeseries.csv'.")


C:\Users\fhung2\AppData\Local\Temp\ipykernel_48124\2412236192.py:92: FutureWarning: 'AS' is deprecated and will be removed in a future version, please use 'YS' instead.
  return pd.date_range(start=start, periods=n, freq="AS")



=== BHA ===
Monthly series: 94  |  Annual series: 24
- BHA_BlueMesa.Elev.txt: Monthly  (444 points from 2024-01-01)
- BHA_BlueMesa.Inflow.txt: Monthly  (444 points from 2024-01-01)
- BHA_BlueMesa.Outflow.txt: Monthly  (444 points from 2024-01-01)
- BHA_BlueMesa.Storage.txt: Monthly  (444 points from 2024-01-01)
- BHA_ComputedStateDepletions.ArizonaActual.txt: Monthly  (444 points from 2024-01-01)
- BHA_ComputedStateDepletions.CaliforniaActual.txt: Monthly  (444 points from 2024-01-01)
- BHA_ComputedStateDepletions.ColoradoActual.txt: Monthly  (444 points from 2024-01-01)
- BHA_ComputedStateDepletions.NevadaActual.txt: Monthly  (444 points from 2024-01-01)
- BHA_ComputedStateDepletions.NewMexicoActual.txt: Monthly  (444 points from 2024-01-01)
- BHA_ComputedStateDepletions.UtahActual.txt: Monthly  (444 points from 2024-01-01)
- BHA_ComputedStateDepletions.WyomingActual.txt: Monthly  (444 points from 2024-01-01)
- BHA_Crystal.Elev.txt: Monthly  (444 points from 2024-01-01)
- BHA_Crystal

In [4]:
# Save files to CommonFiles folder with columns as scenarios
import os
import re
import pandas as pd

# ========= Settings =========
GROUP_ROOT = "CommonFiles"     # where your metric folders live
ONLY_EXT = {".txt"}            # set to None to include all files
# ============================

def read_text(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def parse_values_after_series_slot(text):
    """
    Find '# Series Slot:' line and parse subsequent numeric lines.
    """
    m = re.search(r"^\s*#\s*Series\s+Slot:.*?(?:\r?\n)(.*)$", text, flags=re.S | re.M)
    if not m:
        return []
    vals = []
    for line in m.group(1).splitlines():
        s = line.strip()
        if not s or s.startswith("#"):
            continue
        try:
            vals.append(float(s))
        except ValueError:
            # ignore non-numeric lines
            pass
    return vals

def parse_dates(text):
    """
    Extract start_date/end_date in YYYY-MM-DD if present.
    """
    start_match = re.search(r"start_date:\s*(\d{4}-\d{2}-\d{2})", text)
    end_match   = re.search(r"end_date:\s*(\d{4}-\d{2}-\d{2})", text)
    start_date = pd.to_datetime(start_match.group(1)) if start_match else None
    end_date   = pd.to_datetime(end_match.group(1)) if end_match else None
    return start_date, end_date

def months_inclusive(s, e):
    return (e.year - s.year) * 12 + (e.month - s.month) + 1

def years_inclusive(s, e):
    return (e.year - s.year) + 1

def infer_freq(start_date, end_date, n_points):
    """
    Infer 'MS' (monthly) or 'YS' (annual).
    Priority: match start/end range length; otherwise use heuristics.
    """
    if start_date is not None and end_date is not None:
        m = months_inclusive(start_date, end_date)
        y = years_inclusive(start_date, end_date)
        if m == n_points:
            return "MS"
        if y == n_points:
            return "YS"
        # tolerant fallback
        if abs(m - n_points) <= 1:
            return "MS"
        if abs(y - n_points) <= 1:
            return "YS"
    # heuristic fallbacks
    if n_points >= 24:
        return "MS"
    if n_points <= 3:
        return "YS"
    return "MS"

def build_index(start_date, freq, n_points):
    """
    Build DatetimeIndex of length n_points.
    - MS: anchor at the first day of start_date's month
    - YS: anchor at Jan 1 of start_date's year
    """
    if start_date is None:
        raise ValueError("Missing start_date; cannot build time index.")
    if freq == "MS":
        start = pd.Timestamp(start_date.year, start_date.month, 1)
        return pd.date_range(start=start, periods=n_points, freq="MS")
    if freq in ("YS",):  # annual (year start)
        start = pd.Timestamp(start_date.year, 1, 1)
        return pd.date_range(start=start, periods=n_points, freq="YS")
    raise ValueError(f"Unsupported freq: {freq}")

def should_include(path):
    if not os.path.isfile(path):
        return False
    if ONLY_EXT is None:
        return True
    _, ext = os.path.splitext(path)
    return ext.lower() in ONLY_EXT

base_dir = os.getcwd()
group_root_dir = os.path.join(base_dir, GROUP_ROOT)
if not os.path.isdir(group_root_dir):
    raise FileNotFoundError(f"CommonFiles folder not found at: {group_root_dir}")

# iterate metric subfolders (e.g., 'Mead.Elev', 'Powell.Elev', etc.)
for metric in sorted(d for d in os.listdir(group_root_dir)
                     if os.path.isdir(os.path.join(group_root_dir, d))):
    metric_dir = os.path.join(group_root_dir, metric)

    monthly_cols = {}  # scenario -> Series
    annual_cols  = {}  # scenario -> Series
    summary_rows = []

    for fname in sorted(os.listdir(metric_dir)):
        path = os.path.join(metric_dir, fname)
        if not should_include(path):
            continue

        # Expect filenames like 'BHA_Mead.Elev.txt' -> scenario='BHA'
        base_name = os.path.splitext(fname)[0]
        scenario = base_name.split("_", 1)[0] if "_" in base_name else base_name

        text = read_text(path)
        vals = parse_values_after_series_slot(text)
        if not vals:
            summary_rows.append((fname, "N/A", "No data parsed (no '# Series Slot:' or no numeric values)"))
            continue

        start_date, end_date = parse_dates(text)
        freq = infer_freq(start_date, end_date, len(vals))

        try:
            idx = build_index(start_date, freq, len(vals))
        except Exception as e:
            summary_rows.append((fname, "Unknown", f"Index error: {e}"))
            continue

        s = pd.Series(vals, index=idx, name=scenario)

        if freq == "MS":
            monthly_cols[scenario] = s
            summary_rows.append((fname, "Monthly", f"{len(vals)} pts from {idx[0].date()}"))
        elif freq == "YS":
            annual_cols[scenario] = s
            summary_rows.append((fname, "Annual", f"{len(vals)} pts from {idx[0].date()}"))
        else:
            summary_rows.append((fname, "Unknown", f"{len(vals)} pts"))

    # Combine into aligned DataFrames per metric
    monthly_df = pd.concat(monthly_cols.values(), axis=1).sort_index() if monthly_cols else pd.DataFrame()
    annual_df  = pd.concat(annual_cols.values(),  axis=1).sort_index()  if annual_cols  else pd.DataFrame()

    # Save per-metric CSVs inside the same metric folder
    out_monthly = os.path.join(metric_dir, f"{metric}_monthly.csv")
    out_annual  = os.path.join(metric_dir, f"{metric}_annual.csv")
    if not monthly_df.empty:
        monthly_df.to_csv(out_monthly)
    if not annual_df.empty:
        annual_df.to_csv(out_annual)

    # Print summary
    print(f"\n=== {metric} ===")
    print(f"Monthly series: {len(monthly_cols)} | Annual series: {len(annual_cols)}")
    for fname, step, note in summary_rows:
        print(f"- {fname}: {step} ({note})")
    if not monthly_df.empty:
        print(f"→ Saved: {out_monthly}  shape={monthly_df.shape}")
    if not annual_df.empty:
        print(f"→ Saved: {out_annual}   shape={annual_df.shape}")

print("\n✅ Done. Each metric folder in 'CommonFiles' now has <metric>_monthly.csv and/or <metric>_annual.csv (columns are scenarios).")



=== BlueMesa.Elev ===
Monthly series: 3 | Annual series: 0
- BHA_BlueMesa.Elev.txt: Monthly (444 pts from 2024-01-01)
- CCA_BlueMesa.Elev.txt: Monthly (444 pts from 2024-01-01)
- FAHA_BlueMesa.Elev.txt: Monthly (444 pts from 2024-01-01)
→ Saved: C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\CommonFiles\BlueMesa.Elev\BlueMesa.Elev_monthly.csv  shape=(444, 3)

=== BlueMesa.Inflow ===
Monthly series: 3 | Annual series: 0
- BHA_BlueMesa.Inflow.txt: Monthly (444 pts from 2024-01-01)
- CCA_BlueMesa.Inflow.txt: Monthly (444 pts from 2024-01-01)
- FAHA_BlueMesa.Inflow.txt: Monthly (444 pts from 2024-01-01)
→ Saved: C:\Users\fhung2\ASU Dropbox\Feng-wei Hung\CRSS-ABM\Data\NoABM_Conserve1\CommonFiles\BlueMesa.Inflow\BlueMesa.Inflow_monthly.csv  shape=(444, 3)

=== BlueMesa.Outflow ===
Monthly series: 3 | Annual series: 0
- BHA_BlueMesa.Outflow.txt: Monthly (444 pts from 2024-01-01)
- CCA_BlueMesa.Outflow.txt: Monthly (444 pts from 2024-01-01)
- FAHA_BlueMesa.Outflow.txt

In [5]:
# save file to ALL_Timeseries_CSVs folder
import os
import shutil

SOURCE_ROOT = "CommonFiles"                 # root folder in your screenshot
DEST_FOLDER = "All_Timeseries_CSVs"         # destination folder

# create destination folder if it doesn't exist
os.makedirs(DEST_FOLDER, exist_ok=True)

# walk through all subfolders and copy ONLY .csv files
for root, dirs, files in os.walk(SOURCE_ROOT):
    for file in files:
        if file.lower().endswith(".csv"):
            src = os.path.join(root, file)
            dst = os.path.join(DEST_FOLDER, file)

            # overwrite if it already exists
            shutil.copy2(src, dst)
            print(f"Copied CSV: {file}")

print("\n✅ All CSV time-series files collected in:", DEST_FOLDER)


Copied CSV: BlueMesa.Elev_monthly.csv
Copied CSV: BlueMesa.Inflow_monthly.csv
Copied CSV: BlueMesa.Outflow_monthly.csv
Copied CSV: BlueMesa.Storage_monthly.csv
Copied CSV: ComputedStateDepletions.ArizonaActual_monthly.csv
Copied CSV: ComputedStateDepletions.CaliforniaActual_monthly.csv
Copied CSV: ComputedStateDepletions.ColoradoActual_monthly.csv
Copied CSV: ComputedStateDepletions.NevadaActual_monthly.csv
Copied CSV: ComputedStateDepletions.NewMexicoActual_monthly.csv
Copied CSV: ComputedStateDepletions.UtahActual_monthly.csv
Copied CSV: ComputedStateDepletions.WyomingActual_monthly.csv
Copied CSV: Crystal.Elev_monthly.csv
Copied CSV: Crystal.Inflow_monthly.csv
Copied CSV: Crystal.Outflow_monthly.csv
Copied CSV: Crystal.Storage_monthly.csv
Copied CSV: CumulativeNaturalFlow.AboveBlueMesa_monthly.csv
Copied CSV: CumulativeNaturalFlow.AboveImperial_monthly.csv
Copied CSV: CumulativeNaturalFlow.AboveLeesFerry_monthly.csv
Copied CSV: DiagnoseOutput.1_NF_AboveLeesFerry_annual.csv
Copied CS

In [6]:
# Save everything to NOABM folder with prefix
import os
import shutil

SOURCE_ROOT = "CommonFiles"                       # root folder to search
DEST_ROOT   = "All_Timeseries_CSVs"               # main destination folder
NO_ABM_DEST = os.path.join(DEST_ROOT, "NOABM")   # separate subfolder for prefixed files

PREFIX = "NOABM_"                                # prefix for output files

# create destination folders if they don't exist
os.makedirs(NO_ABM_DEST, exist_ok=True)

# walk through all subfolders and copy ONLY .csv files
for root, dirs, files in os.walk(SOURCE_ROOT):
    for file in files:
        if file.lower().endswith(".csv"):
            src = os.path.join(root, file)

            # new name (prepended with prefix)
            new_name = PREFIX + file
            dst = os.path.join(NO_ABM_DEST, new_name)

            shutil.copy2(src, dst)
            print(f"Copied CSV to NO_ABM folder: {new_name}")

print("\n✅ All CSV files copied into:", NO_ABM_DEST)


Copied CSV to NO_ABM folder: NOABM_BlueMesa.Elev_monthly.csv
Copied CSV to NO_ABM folder: NOABM_BlueMesa.Inflow_monthly.csv
Copied CSV to NO_ABM folder: NOABM_BlueMesa.Outflow_monthly.csv
Copied CSV to NO_ABM folder: NOABM_BlueMesa.Storage_monthly.csv
Copied CSV to NO_ABM folder: NOABM_ComputedStateDepletions.ArizonaActual_monthly.csv
Copied CSV to NO_ABM folder: NOABM_ComputedStateDepletions.CaliforniaActual_monthly.csv
Copied CSV to NO_ABM folder: NOABM_ComputedStateDepletions.ColoradoActual_monthly.csv
Copied CSV to NO_ABM folder: NOABM_ComputedStateDepletions.NevadaActual_monthly.csv
Copied CSV to NO_ABM folder: NOABM_ComputedStateDepletions.NewMexicoActual_monthly.csv
Copied CSV to NO_ABM folder: NOABM_ComputedStateDepletions.UtahActual_monthly.csv
Copied CSV to NO_ABM folder: NOABM_ComputedStateDepletions.WyomingActual_monthly.csv
Copied CSV to NO_ABM folder: NOABM_Crystal.Elev_monthly.csv
Copied CSV to NO_ABM folder: NOABM_Crystal.Inflow_monthly.csv
Copied CSV to NO_ABM folder: 

In [7]:
print("\n✅ Done.")


✅ Done.
